In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 利用列名和数据构建dataframe
df = pd.DataFrame({
    'key1':['a', 'a', 'b', 'b', 'a'],
    'key2' : ['one', 'two', 'one', 'two', 'one'],
    'data1' : np.random.randn(5),
    'data2' : np.random.randn(5)
})
df

,key1,key2,data1,data2
0,a,one,-0.223888,0.114470
1,a,two,0.624460,-1.265352
2,b,one,0.371178,1.426521
3,b,two,-1.663985,0.810567
4,a,one,-0.570321,-0.005611


In [3]:
# 利用key1进行分组，并计算data1列的均值
# 方式：访问data1,并根据key1调用groupby
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
# 变量grouped是一个groupby对象，它实际上还没有进行计算，只是含有一些分组键df['key1']的中间数据
# 简言之该对象有了对各分组执行运算的所有信息
grouped.mean()

key1
a   -0.056583
b   -0.646404
Name: data1, dtype: float64

数据根据  分组键  进行了聚合，产生了一个新的series,其索引为key1列中的唯一值

In [6]:
# 一次传入多个数组,通过多个键对数据进行分组，得到具有层次化索引的series
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means

key1  key2
a     one    -0.397105
      two     0.624460
b     one     0.371178
      two    -1.663985
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,-0.397105,0.624460
b,0.371178,-1.663985


In [8]:
# 分组可以是任何长度适当的数组
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

In [9]:
df['data1'].groupby([states,years]).mean()

California  2005    0.624460
            2006    0.371178
Ohio        2005   -0.943936
            2006   -0.570321
Name: data1, dtype: float64

还可以将列名(可以是字符，数字等)作为分组建

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.056583,-0.385498
b,-0.646404,1.118544


In [11]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.397105  0.054429
     two   0.624460 -1.265352
b    one   0.371178  1.426521
     two  -1.663985  0.810567

groupby进行分组时对非数据列（麻烦列）进行了排除，默认情况下所有数值都会被聚合

In [12]:
# size方法返回分组大小
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64